This script is created to get the list of devices from eUDT site. It uses eUDT API with an authorization token
received from UDT Programming Specialist. User has to authenticate authorization token, then use received
access_token to get information from eUDT site. Token is valid for 5 minutes, after that it has to be refreshed.
Refreshing token is valid for 30 minutes, after that whole authorization process has to be done again.

In [ ]:
# import the libraries
import json
import requests
import pprint
import pandas as pd

In [ ]:
def authentication(headers, params, api_url):
    """
    This code is used to authenticate my API Token and receive an access token together with refresh token. Access token is valid for 5 minutes,
    after that it has to be refreshed with usage of refresh token which is valid 30 minutes. Authentication allows user to use eUdt API.
    """
    response = requests.post(api_url, headers = headers, json=params)

    access_token = response.json()["accessToken"]
    authorization = f'Bearer {access_token}'

    refresh_token = response.json()["refreshToken"]

    return authorization, refresh_token

In [ ]:
def logging():
    """
    This code serves for keeping the logging credentials and headers in one place.
    """
    # Opening JSON file
    f = open('udt.json')  # d.paw email   

    # returns JSON object as a dictionary
    data = json.load(f)

    # login credentials
    email = data['email']
    password = data['password']
    token = data['token']

    headers = {
    "Content-Type": "application/json",
    "X-Api-Token": token,
    }

    params = {
    "login": email,
    "password": password
    }

    return headers, params, token

In [ ]:
def get_access(api_url):
    """
    This code is created to get access to eUDT api.
    """
    headers, params, token = logging()

    authorization, refresh_token = authentication(headers, params, api_url)

    return authorization, refresh_token, token

In [ ]:
def refresh_the_token(api_url, refresh_token, token):
    """
    This code is used to refresh access_token. It is valid for 30 minutes, after that main token has to be authenticated again.
    """
    headers = {
    "Content-Type": "application/json",
    "X-Api-Token": token,
    }
    params = {
        "refreshToken" : refresh_token
    }

    response = requests.put(api_url, headers = headers, json=params)
    refreshed_access_token = response.json()['accessToken']
    authorization = f'Bearer {refreshed_access_token}'
    
    return authorization, refreshed_access_token

In [ ]:
def open_site(authorization, url=None, querystring=""):
    """
    This code is created to take information about the devices from eUDT site.
    """
    payload = ""
    headers = {
    "authority": "api.eudt.gov.pl",
    "accept": "application/json, text/plain, */*",
    "accept-language": "pl,en;q=0.9,en-GB;q=0.8,en-US;q=0.7",
    "cache-control": "no-cache",
    "cookie": "",
    "expires": "0",
    "origin": "https://eudt.gov.pl",
    "pragma": "no-cache",
    "sec-ch-ua": "^\^Chromium^^;v=^\^112^^, ^\^Microsoft",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "^\^Windows^^",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "x-context": "32283",
    "x-xsrf-token": "YmRlM2ZmNThiMGVlYmE2NmUxN2FjNDZlZjkzMmQxOGQyMDk5MGQ0NjdjNjNmOGJjOTJiMTI4MWMxMmJlNzEwODthOGM1YzZlOS0xNTVjLTRmMTQtOTc4Ni1jNDcwYTVjMTg3ZDc=",
    "Authorization" : authorization
    }
    if url:
        url = url
    else:
        url = "https://api.eudt.gov.pl/Device"

    # # getting applications
    # url = "https://api.eudt.gov.pl/Application/values"

    response = requests.request("GET", url, data=payload, headers=headers, params=querystring)

    return response.json()['data']

In [ ]:
def get_devices_data(api_url, refresh_token, token):
    """
    This is the main function which serves to get all the data about the devices that user has access to.
    It takes just 3 positional arguments, api_url is predefined, refresh_token is taken from get_access function,
    token is given from eUDT.
    """
    # take data from eUDT site. Try to get it directly, if not available, then refresh the token, if token is not 
    # valid anymore, authorize it again
    try:
        data = open_site(authorization)
    except:
        try:
            print("refreshing token")
            authorization = refresh_the_token(api_url, refresh_token, token)
            data = open_site(authorization)
        except:
            print("get the new token")
            authorization, refresh_token, token = get_access(api_url)
            data = open_site(authorization)

    # create empty dataframe for keeping data about the devices that the user has access to
    df = pd.DataFrame(columns=['Date', "Operator", "Device", "Device Type", 
                           "Evidence Number", "Factory Number", "Year of Construction",
                           "Number of Charging Points", "Status", "Address"], index=range(0))
    
    pprint.pprint(data)
    
    # loop through every device to get the details about it
    for i in range(len(data)):
        df.loc[i, 'Date'] = data[i]['lastInspectionDate'].replace("T00:00:00", "")
        df.loc[i, 'Device'] = "{} {}".format(data[i]['typeDescription'], data[i]["variantFull"])
        df.loc[i, 'Device Type'] = data[i]['type']
        df.loc[i, 'Evidence Number'] = data[i]['udtNumber']
        df.loc[i, 'Factory Number'] = data[i]['serialNumber']
        df.loc[i, 'Year of Construction'] = data[i]['constructionYear']
        df.loc[i, 'Status'] = data[i]['status']
        df.loc[i, 'Address'] = "{} {} {}".format(data[i]['city'], data[i]['street'], data[i]['streetNo'])
        df.loc[i, 'Operator'] = data[i]['userName']

    # export excel file with all the data
    df.to_excel("List of Devices from UDT.xlsx", index=False)

In [ ]:
def get_invoice_id(api_url, refresh_token, token):
    """
    This is the main function which serves to get all invoices.
    It takes just 3 positional arguments, api_url is predefined, refresh_token is taken from get_access function,
    token is given from eUDT.
    """
    # take data from eUDT site. Try to get it directly, if not available, then refresh the token, if token is not 
    # valid anymore, authorize it again
    url = "https://api.eudt.gov.pl/FinancialDocument"
    querystring = {"sorting.fieldName":"Date","sorting.direction":"2","rowsRange.skip":"0","rowsRange.take":"25"}
    try:
        data = open_site(authorization, url, querystring)
    except:
        try:
            print("refreshing token")
            authorization = refresh_the_token(api_url, refresh_token, token)
            data = open_site(authorization, url, querystring)
        except:
            print("get the new token")
            authorization, refresh_token, token = get_access(api_url)
            data = open_site(authorization, url, querystring)   

    return data 

In [ ]:
def get_data_from_api(api_url, authorization, token):
    headers = {
                "Authorization" : authorization,
                "X-Api-Token": token
                }
    try:
        response = requests.get(api_url, headers=headers)
        response.raise_for_status()
        data = response.content
        return data
    except requests.exceptions.RequestException as e:
        print("Error:", e)
        return None

# if __name__ == "__main__":
#     api_url = "http://api.eudt.gov.pl/FinancialDocument/2205462/File" #2205462
#     data = get_data_from_api(api_url)

#     if data:
#         print("Retrieved data:")
#         print(data)
#     else:
#         print("Failed to retrieve data from the API.")


In [ ]:
def get_invoice(authorization, api_url, refresh_token, token, id):
    """
    This is the main function which serves to download latest invoice.
    It takes just 4 positional arguments, api_url is predefined, refresh_token is taken from get_access function,
    token is given from eUDT, id is a result of get_invoice_id function. Id is variable which contains the id of 
    document which will be downloaded.
    """
    # take data from eUDT site. Try to get it directly, if not available, then refresh the token, if token is not 
    # valid anymore, authorize it again
    url = f"https://api.eudt.gov.pl/FinancialDocument/{id}/File"
  
    # try:
    #     data = open_site2(authorization, url)
    # except:
    try:
        print("refreshing token")
        authorization, refreshed_access_token = refresh_the_token(api_url, refresh_token, token)
        data = get_data_from_api(url, authorization, token)
    except:
        print("get the new token")
        authorization, refresh_token, token = get_access(api_url)
        data = get_data_from_api(url, authorization, token)  

    return data     

In [ ]:
api_url = "https://api.eudt.gov.pl/token"
authorization, refresh_token, token = get_access(api_url)

In [ ]:
# get_devices_data(api_url, refresh_token, token)

In [ ]:
data = get_invoice_id(api_url, refresh_token, token)
id = data[0]['id']

In [ ]:
response = get_invoice(authorization, api_url, refresh_token, token, id)
print(response)

In [ ]:
# Piece of code to export given content to PDF file 
# The provided content
encoded_content = response

output_filename = "decoded.pdf"
with open(output_filename, "wb") as output_file:
    output_file.write(encoded_content)

print(f"Decoded content saved to {output_filename}")